# **API web**

## HTTP

HTTP (protocolo de transferencia de hipertexto) es un protocolo de internet, un conjunto de reglas para transferir datos entre dos sistemas informáticos, generalmente, entre un navegador web y un sitio web. Pero un navegador web no es el único tipo de cliente posible, también puede ser cualquier otro programa, incluido un programa de Python.

## API REST
Los recursos web son excelentes para compartir datos y brindar servicios. Los utilizamos para hacer cálculos, escuchar música, etc. Se supone que un usuario promedio de un servicio no posee ningún conocimiento especial sobre programación, por lo que suele utilizar una interfaz gráfica que le permite interactuar con el servidor (un navegador, en el caso de los recursos web). Pero, ¿y si queremos recuperar algunos datos de la web automáticamente cada hora 24/7? ¿O si necesitamos usar algún servicio como parte de un pipeline de datos más grande? Abrir un navegador y hacer las cosas de forma manual no parece la mejor opción, así que ahí es donde entra en juego la API web.

API significa interfaz de programación de aplicaciones. Especifica cómo un programa (en contraste con una persona) puede interactuar con un servicio o datos. REST (Representational State Transfer) es un conjunto de principios (estilo de arquitectura de software) desarrollado en el año 2000 y ampliamente utilizado para aplicaciones web en la actualidad. Según el REST, hay dos caras: cliente y servidor, que son bastante independientes.

El cliente usa el servidor para acceder a recursos, el URI (Identificador uniforme de recursos) es todo lo que el cliente necesita saber, y toda la magia ocurre del lado del servidor. URI para recursos web se conoce como URL (Localizador uniforme de recursos). Interactúan con solicitudes HTTP autosuficientes, que son livianas y universales. Un cliente realiza su petición llamando a una URL (por tanto, una llamada es todo intento de obtener una respuesta de un servidor), y el servidor devuelve su respuesta.

## Obtener datos con API

Veamos cómo se aplica el método GET en la vida real. Digamos que necesitas información sobre los tipos de cambio actuales para calcular el valor de una cartera de inversiones en USD. ¿Cómo puedes obtener las tarifas de forma automática? En primer lugar, tenemos que encontrar una API web que proporcione datos en respuesta a las solicitudes correspondientes. Comenzaremos con una que no requiere ninguna autenticación.

Aquí está la URL base:  https://api.frankfurter.app. Parece un enlace de internet normal. Visita www.frankfurter.app para saber cómo usarla. Podemos descubrir que /latest nos dirige a las tarifas actuales. La URL final https://api.frankfurter.app/latest se denomina endpoint. El uso básico es:


In [1]:
import requests

response = requests.get("https://api.frankfurter.app/latest")
print(response)

<Response [200]>


Aquí usamos la función get() de la librería requests incorporada para obtener algo del recurso proporcionado. Lo que recibimos fue un objeto response(documentado aquí), que consta de varios conjuntos de información: código de estado, encabezado, contenido y otros. Cuando tratamos de examinar el objeto con print() vemos el código de estado HTTP por defecto. Este muestra cómo se ejecutó la solicitud y consta de tres dígitos: 200 significa que todo está bien, mientras que 4XX y 5XX indican problemas del lado del cliente y del servidor. Definitivamente, queremos acceder a más información, así que vamos a utilizar el método json() para analizar el contenido de la respuesta:

In [12]:
import requests

response = requests.get("https://api.frankfurter.app/latest")

print(response.json())

{'amount': 1.0, 'base': 'EUR', 'date': '2024-02-14', 'rates': {'AUD': 1.6521, 'BGN': 1.9558, 'BRL': 5.3069, 'CAD': 1.4509, 'CHF': 0.9493, 'CNY': 7.7065, 'CZK': 25.348, 'DKK': 7.4537, 'GBP': 0.85258, 'HKD': 8.3754, 'HUF': 388.73, 'IDR': 16717, 'ILS': 3.922, 'INR': 88.96, 'ISK': 148.7, 'JPY': 161.28, 'KRW': 1428.41, 'MXN': 18.3559, 'MYR': 5.1271, 'NOK': 11.3375, 'NZD': 1.7607, 'PHP': 60.164, 'PLN': 4.3415, 'RON': 4.9754, 'SEK': 11.3127, 'SGD': 1.4451, 'THB': 38.76, 'TRY': 32.942, 'USD': 1.0713, 'ZAR': 20.398}}


Esto es mucho más útil para nuestro propósito: vemos el monto, la moneda base y la fecha, seguido de la cantidad de las otras monedas que se puede comprar por este monto. Vamos a reajustar la solicitud. ¿Cuántas libras se pueden comprar con $20? Al leer la documentación en el sitio, podemos aprender que debemos usar los parámetros from (desde), to (hasta) y amount (importe). Para mayor comodidad, requests permite pasar los parámetros de una petición HTTP como un diccionario Python normal:

In [13]:
import requests

params = {"from": "USD", "to": "GBP", "amount": 20}
res = requests.get("https://api.frankfurter.app/latest", params=params)
print(res.json())

{'amount': 20.0, 'base': 'USD', 'date': '2024-02-14', 'rates': {'GBP': 15.9167}}


De forma alternativa, los parámetros pueden ir después del enlace de acuerdo con el siguiente esquema: 
```python
endpoint?param_1=value&param_2=value
```

La siguiente solicitud terminará con el mismo resultado que la anterior:

In [14]:
import requests

res = requests.get("https://api.frankfurter.app/latest?from=USD&to=GBP&amount=20")
print(res.json())

{'amount': 20.0, 'base': 'USD', 'date': '2024-02-14', 'rates': {'GBP': 15.9167}}


Se recomienda el uso del primer método, en el que pasamos parámetros como un diccionario, porque permite evitar la combinación de todos los parámetros en una sola cadena larga y, por lo tanto, hace que el código sea más limpio.

Además del código de estado y del contenido, las respuestas contienen headers (encabezados), que a su vez comprenden metadatos que, en este caso, son datos en protocolo HTTP. Ahí podemos encontrar información técnica sobre la respuesta: versión del servidor, tipo de contenido, programación, fecha, etc.

In [15]:
import requests

response = requests.get("https://api.frankfurter.app/latest")

print(response.headers["Server"])
print(response.headers["Content-Type"])
print(response.headers["Content-Length"])

cloudflare
application/json; charset=utf-8


KeyError: 'content-length'

## Autenticación

El endpoint de los ejemplos anteriores permite obtener datos sin ningún tipo de autenticación, es decir, el cliente no presenta ni verifica su identidad. Eso es bastante fuera de lo común porque muchos recursos web requieren algún tipo de autenticación para obtener datos de ellos o acceder a su funcionalidad más amplia, y muchas aplicaciones también pueden cobrar por su funcionalidad completa. Sin embargo, la mayoría de las APIs permiten obtener cierta información de forma gratuita.

Por lo general, la autenticación requiere la existencia de un secreto mutuo conocido solo por el cliente y el recurso web. Puede ser el clásico ejemplo de nombre de usuario y contraseña, o bien una clave API o un token. La clave API es como una contraseña, pero sin nombre de usuario. A menudo, es una cadena criptográfica similar a:

```python
0aebfa14099649068fcddcac9eabbed8
```

Se considera secreta debido a la gran dificultad que supondría adivinar su valor exacto; hay demasiadas variantes como para intentar adivinarla aleatoriamente. Por eso las claves API solo se comparten entre aquellas personas o sistemas que están estrictamente obligados a conocerla. Compartirla públicamente acabaría con su seguridad.


In [7]:
import requests

api_key = "Nt5WVU0p0aTAyqpDVFUCxBUyYt2XVNBi" # Esto es un ejemplo, no es real
params = {'apikey':api_key, 'q':'Lima'}

aw_location_url = "https://dataservice.accuweather.com/locations/v1/cities/search"
aw_location_res = requests.get(aw_location_url, params=params)


In [8]:
import pprint

pprint.pprint(aw_location_res.json())

[{'AdministrativeArea': {'CountryID': 'PE',
                         'EnglishName': 'Lima',
                         'EnglishType': 'Municipality',
                         'ID': 'LMA',
                         'Level': 1,
                         'LocalizedName': 'Lima',
                         'LocalizedType': 'Municipality'},
  'Country': {'EnglishName': 'Peru', 'ID': 'PE', 'LocalizedName': 'Peru'},
  'DataSets': ['AirQualityCurrentConditions',
               'AirQualityForecasts',
               'FutureRadar',
               'MinuteCast',
               'Radar',
               'TidalForecast'],
  'EnglishName': 'Lima',
  'GeoPosition': {'Elevation': {'Imperial': {'Unit': 'ft',
                                             'UnitType': 0,
                                             'Value': 344.0},
                                'Metric': {'Unit': 'm',
                                           'UnitType': 5,
                                           'Value': 105.0}},
            

In [9]:
for loc_info in aw_location_res.json():
    print(
        "{:>8}   {:10}   {:16}    {:16}".format(
            loc_info["Key"],
            loc_info["EnglishName"],
            loc_info["Country"]["EnglishName"],
            loc_info["AdministrativeArea"]["EnglishName"],
        )
    )

  264120   Lima         Peru                Lima            
  330111   Lima         United States       Ohio            
    7814   Lima         Argentina           Buenos Aires    
  120538   Lima         Cuba                Pinar del Río   
  119186   Lima         Cuba                Artemisa        
  220597   Lima         Jamaica             Saint James     
 2242734   Lima         United States       Pennsylvania    
 2162324   Lima         United States       Oklahoma        
 2126310   Lima         United States       New York        
  339180   Lima         United States       Montana         
 2241358   Lima         United States       Illinois        
  307487   Lima         Sweden              Dalarna         
  260683   Lima         Paraguay            San Pedro       
 3416952   Lima         Philippines         Leyte           
 3580837   Lima         Mexico              Oaxaca          
 2301239   Lima         Brazil              Ceará           
 2301215   Lima         

¡Vaya, sí que hay varias ciudades de Lima! Digamos que nos interesa la New York de la primera línea. Ahora, al conocer la clave de ubicación, podemos consultar el pronóstico para esa ubicación llamando a un endpoint diferente desde AccuWeather Forecast API.

In [10]:
import requests

api_key = "Nt5WVU0p0aTAyqpDVFUCxBUyYt2XVNBi"
params = {"apikey": api_key, "metric": True}

location_id = 264120
aw_forecast_url = "https://dataservice.accuweather.com/forecasts/v1/daily/5day/" + str(
    location_id
)
aw_forecast_res = requests.get(aw_forecast_url, params=params)

El resultado también es una estructura JSON extensa . Como en el caso anterior, podemos imprimir y analizarla entera, pero nos ahorraremos esta parte y nos limitaremos a imprimir los campos que más nos interesen.

In [11]:
for daily_forecast in aw_forecast_res.json()["DailyForecasts"]:
    print(
        "{}   {:30} {}{}  {}{}".format(
            daily_forecast["Date"],
            daily_forecast["Day"]["IconPhrase"],
            daily_forecast["Temperature"]["Minimum"]["Value"],
            daily_forecast["Temperature"]["Minimum"]["Unit"],
            daily_forecast["Temperature"]["Maximum"]["Value"],
            daily_forecast["Temperature"]["Maximum"]["Unit"],
        )
    )

2024-02-15T07:00:00-05:00   Intermittent clouds            23.5C  28.4C
2024-02-16T07:00:00-05:00   Intermittent clouds            23.4C  28.6C
2024-02-17T07:00:00-05:00   Intermittent clouds            23.3C  28.2C
2024-02-18T07:00:00-05:00   Partly sunny                   23.2C  28.2C
2024-02-19T07:00:00-05:00   Partly sunny                   23.4C  28.6C


# **Ejercicios**

**Ejercicio 1**

La API web es una herramienta versátil utilizada por más que solo empresas: hay muchos museos, bibliotecas, proyectos científicos y entusiastas que ponen su información y servicios a disposición del público en general. Utiliza la librería requests y el endpoint proporcionado en la url para obtener información sobre una pintura de la colección del Museo Metropolitano. Examina la URL: el enlace base es https://collectionapi.metmuseum.org/, luego nos dirigimos a un recurso específico. En algún momento hay una parte v1. Esta es la versión de la API. Las diferentes versiones pueden actuar de manera diferente, por lo que es común reflejar la actual. Finalmente, el número 437133 es la identificación de la pintura.

- Importa requests.
- Llama al endpoint mediante la url proporcionada. Recupera el resultado en formato JSON aplicándole el método apropiado. Guarda la JSON resultante en la variable response.
- Muestra artistDisplayName desde JSON llamándolo por la clave.

In [17]:
import requests

base_url = "https://collectionapi.metmuseum.org/"
url = base_url + "public/collection/v1/objects/437133"

response = requests.get(url)
print(response.json()['artistDisplayName'])

Claude Monet


**Ejercicio 2**

Para llegar al endpoint public/collection/v1/departments utilizando la misma URL base, simplemente agrega /public/collection/v1/departments a la URL base. Guarda la URL resultante en la variable url.

A continuación, en el precódigo, verás una línea donde almacenamos la respuesta del endpoint y la guardamos en la variable response.
Tu objetivo ahora es convertir la respuesta en un archivo JSON. Luego, itera sobre response.json()['departments'] y muestra los que tienen 'Art' en dpt['displayName'].

In [18]:
import requests

base_url = "https://collectionapi.metmuseum.org/"
url = base_url + "public/collection/v1/departments"  
response = requests.get(url)

for dpt in response.json()["departments"]:
    if "Art" in dpt["displayName"]:  
        print(dpt)

{'departmentId': 1, 'displayName': 'American Decorative Arts'}
{'departmentId': 3, 'displayName': 'Ancient Near Eastern Art'}
{'departmentId': 5, 'displayName': 'Arts of Africa, Oceania, and the Americas'}
{'departmentId': 6, 'displayName': 'Asian Art'}
{'departmentId': 10, 'displayName': 'Egyptian Art'}
{'departmentId': 12, 'displayName': 'European Sculpture and Decorative Arts'}
{'departmentId': 13, 'displayName': 'Greek and Roman Art'}
{'departmentId': 14, 'displayName': 'Islamic Art'}
{'departmentId': 17, 'displayName': 'Medieval Art'}
{'departmentId': 21, 'displayName': 'Modern Art'}


**Ejercicio 3**

En esta tarea, utilizarás parámetros de solicitud. La URL de esta tarea se ha guardado en la variable url. Tu objetivo es establecer params = {'limit': 3} cuando utilices el método get(). Esto permitirá recuperar solo las tres primeras entradas. Almacena la respuesta en la variable response, luego conviértela a JSON y muéstrala.



In [19]:
import requests

url = "https://dummyjson.com/products"

params = {"limit": 3}  
response = requests.get(url, params=params)  
print(response.json()) 

{'products': [{'id': 1, 'title': 'iPhone 9', 'description': 'An apple mobile which is nothing like apple', 'price': 549, 'discountPercentage': 12.96, 'rating': 4.69, 'stock': 94, 'brand': 'Apple', 'category': 'smartphones', 'thumbnail': 'https://cdn.dummyjson.com/product-images/1/thumbnail.jpg', 'images': ['https://cdn.dummyjson.com/product-images/1/1.jpg', 'https://cdn.dummyjson.com/product-images/1/2.jpg', 'https://cdn.dummyjson.com/product-images/1/3.jpg', 'https://cdn.dummyjson.com/product-images/1/4.jpg', 'https://cdn.dummyjson.com/product-images/1/thumbnail.jpg']}, {'id': 2, 'title': 'iPhone X', 'description': 'SIM-Free, Model A19211 6.5-inch Super Retina HD display with OLED technology A12 Bionic chip with ...', 'price': 899, 'discountPercentage': 17.94, 'rating': 4.44, 'stock': 34, 'brand': 'Apple', 'category': 'smartphones', 'thumbnail': 'https://cdn.dummyjson.com/product-images/2/thumbnail.jpg', 'images': ['https://cdn.dummyjson.com/product-images/2/1.jpg', 'https://cdn.dummy